In [2]:
import pandas as pd
import numpy as np
import time
import cantera as ct
from matplotlib import pyplot as plt
import csv
import math
import os
import sys
from IPython.display import Image

import itertools
import logging
from collections import defaultdict

import scipy.linalg as la

print("Running Cantera version: {}".format(ct.__version__))

Running Cantera version: 2.4.0


### load model 

In [3]:
# Grabow model and RMG input files
# cti_file_rmg = 'Grabow_species_removed/chem_annotated_removed.cti'
cti_file_rmg = 'chem_annotated.cti'

# Multipliers
cat_mult = 1

#constants
pi = math.pi

# set initial temps, pressures, concentrations
# temp = 230+273.15         # [kelvin]
temp = 528# 483.5                # [kelvin]
pressure = 15.3*1e5         # [Pascals] 
X_co = 0.025
X_co2= 0.025
X_h2 = 0.95
mw_co = 28.01e-3     # [kg/mol]
mw_co2 = 44.01e-3    # [kg/mol]
mw_h2 = 1.01e-3      # [kg/mol]

# CO/CO2/H2/H2O: typical is 
concentrations_rmg = {'CO(3)': X_co,'CO2(4)': X_co2, 'H2(2)':X_h2}

# initialize cantera gas and surface
gas= ct.Solution(cti_file_rmg,'gas')

# surf_grab = ct.Interface(cti_file,'surface1_grab', [gas_grab])
surf = ct.Interface(cti_file_rmg,'surface1', [gas])

# gas_grab.TPX = 
gas.TPX = temp, pressure, concentrations_rmg
surf.TP = temp, pressure
surf.coverages = 'X(1):1.0'

# create gas inlet
inlet = ct.Reservoir(gas)

#create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume
rradius = 35e-3
rlength = 70e-3
rvol = (rradius**2)*pi*rlength

# Catalyst Surface Area
site_density = surf.site_density*1000                #[mol/m^2]cantera uses kmol/m^2, convert to mol/m^2
cat_weight = 4.24e-3                                 #[kg]
cat_site_per_wt = (300*1e-6)*1000                    #[mol/kg] 1e-6mol/micromole, 1000g/kg
cat_area = cat_mult*site_density/(cat_weight*cat_site_per_wt) #[m^3]

# reactor initialization
r = ct.IdealGasConstPressureReactor(gas, energy='off')
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol

# flow controllers (Graaf measured flow at 293.15 and 1 atm)
one_atm = ct.one_atm
FC_temp = 293.15
volume_flow = 4.24e-3                                               # [m^3/s]
molar_flow = volume_flow*one_atm/(8.3145*FC_temp)                     # [mol/s]
mass_flow = molar_flow*(X_co*mw_co+X_co2*mw_co2) # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# initialize reactor network
sim = ct.ReactorNet([r])
# set relative and absolute tolerances on the simulation
sim.rtol = 1.0e-11
sim.atol = 1.0e-22

#for debugging
sim.verbose = True
sim.reinitialize()


print(cat_area)

Initializing reactor network.
Reactor 0: 64 variables.
              0 sensitivity params.
Number of equations: 64
Maximum time step:                0
0.02313679245283019


### generate matrices